In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from datetime import datetime
import re
import warnings
import math

In [2]:
warnings.simplefilter('ignore')

## Flightplan

In [ ]:
arr_df = pd.DataFrame()
dep_df = pd.DataFrame()

In [ ]:
daterange = pd.date_range(start='12/22/2019', end='12/28/2019').strftime('%Y%m%d')

In [ ]:
for date in daterange.tolist():
    arr = pd.read_excel(f'../data/flightplan/arr_{date}.xlsx', header = 0)
    dep = pd.read_excel(f'../data/flightplan/dep_{date}.xlsx', header = 0)
    datee = datetime.strptime(date,'%Y%m%d').strftime('%Y-%m-%d')
    arr['SDT'] = datee
    dep['SDT'] = datee
    arr_df = pd.concat([arr_df, arr])
    dep_df = pd.concat([dep_df, dep])

In [ ]:
arr_df = arr_df.drop('Seq', axis = 1)
dep_df = dep_df.drop('Seq', axis = 1)

In [ ]:
arr_df.to_csv('../input/arr_fp_20191222-20191228.csv')
dep_df.to_csv('../input/dep_fp_20191222-20191228.csv')

In [ ]:
# ACDM하고 데이터를 합칠 수 있을까?? -> 활주로 정보 넣기
# Traj0501 = io.loadmat(data_file)


## Takeoff Distance

T/O Distance 

- OAT, Pressure Altitude <br>
	-> METAR에서 가져올 수 있을 듯
	-> Density Altitude를 구하고 변수로 넣기

- Gross Weight <br>
	-> 이게 애매하단 말이지...
	-> 기종 별로는 할 수 있음

- Wind components <br>
	-> METAR에서 가져오기
	-> Headwind, Tailwind 성분만 가지고 하기

- RWY condition <br>
	-> METAR에서 가져오기
	-> Wet rwy 
	-> RWY slope는 rksi = 0


*** T/O speed varies with T/O Weight

### METAR

In [3]:
metar_data = pd.read_excel('../data/METAR_20191222-20191228.xlsx', header = 0)

In [4]:
# pressure column만들기

metar_data['Pres'] = np.zeros(len(metar_data))
pattern_pres = 'Q[0-9][0-9][0-9][0-9]'

for i in range(len(metar_data)):
    pres = re.search(pattern_pres, metar_data['Original Message'][i])
    metar_data['Pres'][i] = int(pres.group()[1:])

In [5]:
# dewpoint column 만들기

metar_data['Due'] = np.zeros(len(metar_data))
pattern_due = '/..[0-9]|/.[0-9]'

for i in range(len(metar_data)):
    due = re.search(pattern_due, metar_data['Original Message'][i])
    if due.group()[1] == 'M':
        due = int(due.group()[2:]) * (-1)
    else:
        due = int(due.group()[1:])
        
    metar_data['Due'][i] = due

In [6]:
# density altitude 계산하기

metar_data['DA'] = np.zeros(len(metar_data))
fe = 7    # field elevation in meters

for i in range(len(metar_data)):
    pres = metar_data['Pres'][i]
    temp = metar_data['Temp'][i]
    due = metar_data['Due'][i]

    # station pressure (https://www.weather.gov/media/epz/wxcalc/stationPressure.pdf)
    station_pressure_inHg = pres * 0.0295300  * (((288 - (0.0065 * fe))/288)**(5.2561))
    station_pressure_mil = station_pressure_inHg * 33.8639

    # density altitude (https://www.weather.gov/media/epz/wxcalc/densityAltitude.pdf)
    vapor_pressure = 6.11 * (10 **((7.5 * due) / (237.7 + due)))
    tv_kelvin = (temp + 273.15) / (1 - ((vapor_pressure / station_pressure_mil) * (1 - 0.622)))  # Kelvin
    tv_rankine = (tv_kelvin * 9 / 5)
    da = 145366 * ( 1 - ((17.326 * station_pressure_mil * 0.029530 / tv_rankine )**(0.235)))
    
    metar_data['DA'][i] = da


In [7]:
# Headwind Components - 아마 바람 방향 바뀌면 바로 활주로 방향 바꾸니까 그냥 cos 때리자

# rksi rwy15,16 true bearing = 144.66 degree

metar_data['Headwind'] = np.zeros(len(metar_data))
metar_data['Crosswind'] = np.zeros(len(metar_data))

pattern_wind = '[0-9]...[0-9]KT'

for i in range(len(metar_data)):
    wind = re.search(pattern_wind, metar_data['Original Message'][i])
    metar_data['WD'][i] = int(wind.group()[0:3])
    deg = metar_data['WD'][i] - 144.66
    headwind = metar_data['WS'][i] * np.cos(deg * math.pi/180)
    crosswind = metar_data['WS'][i] * np.sin(deg * math.pi/180)
    metar_data['Headwind'][i] = headwind
    metar_data['Crosswind'][i] = abs(crosswind)

In [8]:
# Wet Runway
# 현천을 표시 안한 METAR가 많네...
# 비, 눈 오면 wet으로 1로 넣기 ...... dz는?? 

metar_data['Wet'] = np.zeros(len(metar_data))
pattern_wc = 'CAVOK|NSW|TS|DZ|RA|SN|SG|IC|PL|GR|GS|BR|FG|FU|VA|DU|SA|HZ|SQ|FC|SS|DS'

for i in range(len(metar_data)):
    wc = re.search(pattern_wc, metar_data['Original Message'][i])
    # metar_data['Wet'][i] = wc.group()


In [10]:
metar_data.to_csv('../input/metar_20191222-20191228.csv')

In [ ]:
# DA in feets

"""
pres = 1016.2552
temp = 24
due = 23
fe = 17

# station pressure (https://www.weather.gov/media/epz/wxcalc/stationPressure.pdf)
station_pressure_inHg = pres * 0.0295300  * (((288 - (0.0065 * fe))/288)**(5.2561))
station_pressure_mil = station_pressure_inHg * 33.8639

# density altitude (https://www.weather.gov/media/epz/wxcalc/densityAltitude.pdf)
vapor_pressure = 6.11 * (10 **((7.5 * due) / (237.7 + due)))
tv_kelvin = (temp + 273.15) / (1 - ((vapor_pressure / station_pressure_mil) * (1 - 0.622)))  # Kelvin
tv_rankine = (tv_kelvin * 9 / 5)
da = 145366 * ( 1 - ((17.326 * station_pressure_mil * 0.029530 / tv_rankine )**(0.235)))
"""